In [1]:
# Import required modules
from copy import copy

import cupy as cp
import time

from lcode2dPy.plasma3d_gpu.data import GPUArraysView

from lcode2dPy.plasma3d_gpu.initialization import init_plasma
from lcode2dPy.plasma3d_gpu.solver import Plane2d3vPlasmaSolver

from lcode2dPy.config.default_config import default_config

In [2]:
# Create a config
config = copy(default_config)

config.update({
    'geometry': '3d',
    'processing-unit-type': 'gpu',
    'window-width-step-size': 0.02,
    'window-width': 22,

    'window-length': 1500,
    'xi-step': 0.02,

    'time-limit': 1.5,
    'time-step': 1,

    'plasma-particles-per-cell': 4,
})

# Make ready a plasma solver
solver = Plane2d3vPlasmaSolver(config)

In [3]:
# Create a transversal grid and a beam charge distribution
grid_steps     = config.getint('window-width-steps')
grid_step_size = config.getfloat('window-width-step-size')

grid = ((cp.arange(grid_steps) - grid_steps // 2)
        * grid_step_size)
xs, ys = grid[:, None], grid[None, :]

def rho_b(xi):
    COMPRESS, BOOST, SIGMA, SHIFT = 1, 1, 1, 0
    if xi < -2 * cp.sqrt(2 * cp.pi) / COMPRESS or xi > 0:
        return 0.
        # return np.zeros_like(xs)
    r = cp.sqrt(xs**2 + (ys - SHIFT)**2)
    return (.05 * BOOST * cp.exp(-.5 * (r / SIGMA)**2) *
            (1 - cp.cos(xi * COMPRESS * cp.sqrt(cp.pi / 2))))

In [4]:
import os
import matplotlib.pyplot as plt

def ro_diagnostics(xi, ro):
    if not os.path.isdir('transverse'):
        os.mkdir('transverse')

    fname = f'ro_{xi:+09.2f}.png' if xi else 'ro_-00000.00.png'
    plt.imsave(os.path.join('transverse', fname), ro.T,
               origin='lower', vmin=-0.1, vmax=0.1, cmap='bwr')

    if not os.path.isdir('transverse_noise'):
        os.mkdir('transverse_noise')

    fname = f'ro_{xi:+09.2f}.png' if xi else 'ro_-00000.00.png'
    plt.imsave(os.path.join('transverse_noise', fname), ro.T,
               origin='lower', vmin=-0.01,
               vmax=0.01, cmap='bwr')

In [5]:
import numpy as np

In [6]:
# Create the first plasma slice
fields, particles, currents, const_arrays = init_plasma(config)

# Simulation loop:
start_time = time.time()
xi_step_size = config.getfloat('xi-step')
xi_steps = int(config.getfloat('window-length') / xi_step_size)
ez = []
xi_arr = []

currents_prev = currents.copy()
ro_beam_prev = 0

gpu_index = 0

with cp.cuda.Device(gpu_index):
    for xi_i in range(xi_steps + 1):
        xi = - xi_i * xi_step_size
        
        ro_beam = rho_b(xi)
        ro_beam_prev = rho_b(xi + xi_step_size)

        particles, fields, currents, currents_prev = solver.step_dxi(
            particles, fields, currents, currents_prev, const_arrays, ro_beam,
            ro_beam_prev
        )

        view_fields = GPUArraysView(fields)
        ez.append(view_fields.Ez[grid_steps // 2, grid_steps // 2])
        xi_arr.append(xi)
        if xi % 1. == 0:
            print(f'xi={xi:+.4f} {ez[-1]:+.4e}')
            
        if xi % 25. == 0:
            view_currents = GPUArraysView(currents)
            ro_diagnostics(xi, view_currents.ro)
            np.savez('log', xi=xi_arr, Ez_00=ez)
    
print("--- %s seconds ---" % (time.time() - start_time))

/home/ivan/.local/lib/python3.8/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/home/ivan/Desktop/lcode/lcode2dPy/lcodePy/lcode2dPy/plasma3d_gpu/initialization.py:65: RuntimeWarning: divide by zero encountered in true_divide
  mul = 1 / (lambda_i + lambda_j)  # WARNING: zero division in mul[0, 0]!


xi=+0.0000 +1.0270e-20
xi=-1.0000 -5.3029e-03
xi=-2.0000 -2.7796e-02
xi=-3.0000 -3.7318e-02
xi=-4.0000 +6.8653e-03
xi=-5.0000 +6.5034e-02
xi=-6.0000 +6.2359e-02
xi=-7.0000 +8.4830e-03
xi=-8.0000 -5.1637e-02
xi=-9.0000 -7.1374e-02
xi=-10.0000 -2.1755e-02
xi=-11.0000 +5.2282e-02
xi=-12.0000 +7.0119e-02
xi=-13.0000 +2.6684e-02
xi=-14.0000 -3.6459e-02
xi=-15.0000 -7.2358e-02
xi=-16.0000 -4.2355e-02
xi=-17.0000 +3.4156e-02
xi=-18.0000 +7.2561e-02
xi=-19.0000 +4.3255e-02
xi=-20.0000 -1.9020e-02
xi=-21.0000 -6.7533e-02
xi=-22.0000 -5.8447e-02
xi=-23.0000 +1.2299e-02
xi=-24.0000 +6.8976e-02
xi=-25.0000 +5.7077e-02
xi=-26.0000 -4.4696e-04
xi=-27.0000 -5.7748e-02
xi=-28.0000 -6.8707e-02
xi=-29.0000 -1.0959e-02
xi=-30.0000 +5.9108e-02
xi=-31.0000 +6.7047e-02
xi=-32.0000 +1.8142e-02
xi=-33.0000 -4.4048e-02
xi=-34.0000 -7.2653e-02
xi=-35.0000 -3.3003e-02
xi=-36.0000 +4.3379e-02
xi=-37.0000 +7.2134e-02
xi=-38.0000 +3.5634e-02
xi=-39.0000 -2.7547e-02
xi=-40.0000 -7.0507e-02
xi=-41.0000 -5.1455e-02
xi

In [11]:
# Should work correctly. Compare results with:
# xi         Ez_00
# xi=+0.0000 +0.0000e+00
# xi=+0.0200 -1.4490e-07
# xi=+0.0400 -7.2443e-07
# xi=+0.0600 -2.0278e-06
# xi=+0.0800 -4.3436e-06
# xi=+0.1000 -7.9592e-06
# xi=+0.1200 -1.3161e-05
# xi=+0.1400 -2.0233e-05
# xi=+0.1600 -2.9458e-05
# xi=+0.1800 -4.1116e-05
# xi=+0.2000 -5.5486e-05
# xi=+0.2200 -7.2842e-05
# xi=+0.2400 -9.3456e-05
# xi=+0.2600 -1.1760e-04
# xi=+0.2800 -1.4553e-04
# xi=+0.3000 -1.7751e-04
# xi=+0.3200 -2.1381e-04
# xi=+0.3400 -2.5467e-04
# xi=+0.3600 -3.0033e-04
# xi=+0.3800 -3.5105e-04
# xi=+0.4000 -4.0706e-04
# xi=+0.4200 -4.6859e-04
# xi=+0.4400 -5.3587e-04
# xi=+0.4600 -6.0912e-04
# xi=+0.4800 -6.8855e-04
# xi=+0.5000 -7.7437e-04